# Fit bivariate Matern to empirical (cross-) semivariograms

In [ ]:
import sys

sys.path.insert(0, "../../source")


In [ ]:
import pickle
import xarray as xr

import fields
import model
import plot


In [ ]:
# maximum distance between pairs of points used in variograms
MAX_DIST = 1000
# number of bins in variograms
N_BINS = 30
# optimization bounds for model parameters
PARAM_BOUNDS = dict(
    rho=(1e-8, 1.0),
    nu=[(0.3, 0.95), (0.5, 1.5), (0.3, 0.95)],
    len_scale=(10, 300),
)

# data selection
YEAR = 2021
MONTH = 4
# XCO2 months ahead of SIF
OFFSET = 1

year_month_str = f"{YEAR}0{MONTH}" if MONTH < 10 else f"{YEAR}{MONTH}"


In [ ]:
month_xco2 = MONTH + OFFSET
date_sif = f"{YEAR}-{MONTH}-01"
date_xco2 = f"{YEAR}-{month_xco2}-01"

with xr.open_dataset(
    "../../data/intermediate/OCO2_005deg_months2021_north_america_with_basis.nc4"
) as ds:
    basis_vars = [x for x in list(ds.keys()) if x.startswith("B")]
    ds_sif = (
        ds[["sif", "sif_var"] + basis_vars]
        .sel(time=f"{YEAR}-{MONTH}")
        .squeeze()
        .drop_vars(["B1", "B10", "B20"])
    )
    ds_xco2 = ds[["xco2", "xco2_var"] + basis_vars].sel(time=f"{YEAR}-{month_xco2}").squeeze()


In [ ]:
ds_sif


In [ ]:
ds_xco2


In [ ]:
title = f"OCO-2 SIF Lite v10r, 0.05-Degree Month Average {date_sif}"
units = "SIF 740nm [W/m$^2$/sr/$\mu$m]"
plot.plot_da(ds_sif["sif"], vmin=0, robust=True, title=title, label=units)


In [ ]:
title = f"OCO-2 XCO$_2$ Lite v10r, 0.05-Degree Month Average {date_xco2}"
units = "XCO$_2$ [ppm]"
plot.plot_da(ds_xco2["xco2"], robust=True, title=title, label=units)


In [ ]:
# modeling setup
datasets = [ds_sif, ds_xco2]
timedeltas = [0, OFFSET]

vario_config = fields.VarioConfig(MAX_DIST, N_BINS, n_procs=2)
mf = fields.MultiField(datasets, date_sif, timedeltas)
plot.plot_fields(mf, ["SIF", "XCO2"])


In [ ]:
df_sif_res = mf.fields[0].to_dataframe()[["lon", "lat", "sif"]]
df_sif_res["sif"].describe()


In [ ]:
# construct the empirical (cross-) semivariograms
gamma = mf.empirical_variograms(vario_config)
gamma.df.to_csv(f"../../data/intermediate/models/{year_month_str}/variograms.csv")


In [ ]:
# fit the model via multivariate WLS
bivariate_matern = model.MultivariateMatern(n_procs=2)
bivariate_matern.params.set_bounds(**PARAM_BOUNDS)
bivariate_matern.fit(gamma)
df_params = bivariate_matern.params.to_dataframe()
df_params.to_csv(f"../../data/intermediate/models/{year_month_str}/bivariate_model.csv")
df_params


In [ ]:
title = (
    f"(Cross-) Semivariograms for SIF and XCO$_2$ Residuals, 0.05-Degree Resolution, "
    f"North America, {date_sif}"
)
plot.plot_variograms(bivariate_matern.fit_result, ["SIF", "XCO$_2$"], title=title)


## Save the fields, semivariograms, and model

In [ ]:
with open(f"../../data/intermediate/models/{year_month_str}/fields.pickle", "wb") as f:
    pickle.dump(mf, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(f"../../data/intermediate/models/{year_month_str}/variograms.pickle", "wb") as f:
    pickle.dump(gamma, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(f"../../data/intermediate/models/{year_month_str}/bivariate_model.pickle", "wb") as f:
    pickle.dump(bivariate_matern, f, protocol=pickle.HIGHEST_PROTOCOL)
